[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeyerBender/spatialproteomics_workshop/blob/main/notebooks/workshop01_task.ipynb)

# Analysis of highly multiplexed fluorescence images with spatialproteomics
In this notebook, we will learn how to analyze highly multiplexed fluorescence images using the `spatialproteomics` package.
This tutorial will illustrate all major processing steps that are required to extract insights from these complex images.

These include the following:
1. Reading in a highly multiplexed image and creating a spatialproteomics object
2. Performing basic image processing to boost the signal-to-noise ratio
3. Performing nuclear segmentation using _cellpose_
4. Quantifying protein expression per cell
5. Predicting cell types with a simple argmax technique
6. Plotting the results

This tutorial follows the workflow outlined in the [spatialproteomics documentation](https://sagar87.github.io/spatialproteomics/index.html), where you can find more details on individual method.

In [ ]:
# download the data
# if you have already run this cell once, there is no need to run it again

# use this when running on colab
! wget https://www.huber.embl.de/users/matthias/spatialproteomics_workshop_data.tar.gz /content/spatialproteomics_workshop_data.tar.gz
! tar -xzf /content/spatialproteomics_workshop_data.tar.gz
! pip install --quiet spatialproteomics==0.7 cellpose==3.1.0
data_dir = '/content/data'

# use this when running locally
#! wget https://www.huber.embl.de/users/matthias/spatialproteomics_workshop_data.tar.gz spatialproteomics_workshop_data.tar.gz
#! tar -xzf spatialproteomics_workshop_data.tar.gz
#data_dir = 'data'

In [ ]:
# importing all of the required packages
import spatialproteomics as sp
from skimage.io import imread
import matplotlib.pyplot as plt
from scipy.signal import medfilt2d
import os

## 1. Getting Started

Before we can get started with _spatialproteomics_, we first need to read in the image. Highly multiplexed fluorescence images are typically stored in the _tiff_ file format, which can be read in via the _skimage_ or _tifffile_ libraries. In addition, it is common practice to store the list of markers used during imaging in a txt file. Here, we read in the image and the marker list and combine the two into a _spatialproteomics_ object.

Experimental background: this image is a tissue microarray (TMA) of a healthy lymph node.

In [ ]:
# reading in the image
image = imread(os.path.join(data_dir, 'lymph_node.tiff'))

In [ ]:
# TODO: investigate the shape of the image. What are the different dimensions?

At this point, we do not know which channel corresponds to which marker. For this, we also need to read in the marker list, which is stored in the file `marker_list.txt`.

In [ ]:
# opening the file and read it line by line into a list
with open(os.path.join(data_dir, 'marker_list.txt'), 'r') as file:
    lines = file.readlines()

# removing any trailing newline characters from each line
markers = [line.strip() for line in lines]
len(markers)

Great! Now we can put the image and the marker names into a _spatialproteomics_ object.

In [ ]:
sp_object = sp.load_image_data(image, channel_coords=markers)

`sp.load_image_data` returns an xarray object that we can simply inspect by calling in an jupyter cell. Note that the image is stored as the data variable `_image`.

In [ ]:
sp_object

Let's quickly look at what this image looks like. Since it is very difficult to perceive 56 different colors overlaid on top of one another, let's subset our dataset to 5 channels and see what they look like.

In [ ]:
# pp = the preprocessing accessor
# pl = the plotting accessor

# TODO: use the pp accessor to select five channels of your choice. Afterwards, use pl.show() to see what your image looks like.
# Try looking at some different markers.
# Do you spot any interesting patterns? 
# Are all markers equally specific?

## 2. Image Processing
Highly multiplexed fluorescence imaging techniques frequently suffer from poor signal-to-noise ratio. To alleviate this problem, you can threshold out low intensity pixels, thereby boosting the contrast of the image. While there are automated methods to determine the thresholds for such operations, it is difficult to come up with one that works in all cases. Here, we therefore set the thresholds based on manual inspection.

Let's start by looking at the data set in the current form, limiting ourselves to a single channel for now. We can use `pl.autocrop()` and `pl.colorize()` to customize how our image is displayed.

In [ ]:
# select the channel CD4 from the image, and use pl.autocrop() and pl.colorize() to customize your image. Finally, show it with pp.show() just like before.

It looks like CD4 bound to a lot of cells, however there are still some differences in intensities between cells. How do we know that not all of these cells should actually express CD4? CD4 is a marker for T helper cells, and should hence only be expressed on CD3+ T cells. If we look at the combination of CD3 and CD4, we can clearly see that CD4 is not specific to CD3+ cells.

In [ ]:
# TODO: show CD3 and CD4 in the image. Choose two colors which allow you to clearly see the overlap between the two markers

What can we do against this? One option is to apply thresholding, which means that we set every pixel below a certain intensity to 0. Let's do this using the `pp.threshold()` method.

In [ ]:
# TODO: use pp.threshold() to remove the bottom 90% of all pixels and visualize the result. Do you see the difference to before? 
# Also experiment with different thresholds. Can you find one that strikes a good balance between removing the background binding while preserving actual signal?

Let's look at a couple of markers which we can use to predict cell types. We will look for reasonable thresholds for all of them, and then see how the thresholding affected the resulting image.

In [ ]:
# defining some defaults so that our plots all look the same
channels = ['PAX5', 'CD3', 'CD11b', 'CD11c', 'CD15', 'CD68', 'Podoplanin', 'CD31', 'CD34', 'CD90', 'CD56']
quantiles = [0.8, 0.5, 0.8, 0.8, 0.8, 0.8, 0.95, 0.95, 0.95, 0.95, 0.8]
colors = ['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#ffd8b1', '#f58231', '#911eb4', '#fffac8', '#469990', '#fabed4', '#9A6324']

sp_object_thresholded = sp_object.pp[channels].pp.threshold(quantiles)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
# note that we use downsampling here, since colab only provides a limited amount of RAM
# if you have sufficient RAM available, you can remove the downsample step here
_ = sp_object.pp[channels].pp.downsample(4).pl.colorize(colors).pl.show(ax=ax[0])
_ = sp_object_thresholded.pp.downsample(4).pl.colorize(colors).pl.show(ax=ax[1])
ax[0].set_title('Raw')
ax[1].set_title('Thresholded')

Immediately, we can see the structures within the tissue much clearer. So now we have a much less noisy picture, but how can we get from this to a single-cell expression matrix?

## 3. Cell Segmentation

At this point, the only thing missing is a cell segmentation. Since we only have a nuclear marker at hand (and no membrane marker that would stain all cells), we will segment the nuclei and then simply extend the segmentation masks by two pixels in every direction. We are going to use _cellpose_ for this purpose, which is implemented in the tool (tl) module.

In [ ]:
# TODO: use cellpose to segment the channel "DAPI". 
# On colab, you can run this on a GPU by changing your runtime type
# Alternatively, you can also skip this step and read the segmentation we have provided from the file directly.

Looking at the object, you will realize that a new layer called `_segmentation` has appeared. We can use this for downstream analysis and visualization.
We can plot this segmentation over the DAPI channel to see what exactly _cellpose_ did. We can zoom in a little bit to get a clearer picture.

In [ ]:
# plot the DAPI channel, and overlay the segmentation with pl.show(render_segmentation=True)
# try looking at different regions of the tissue. Did the segmentation capture every cell, or do you spot some flaws?

There are two issues with the current masks. One is that sometimes very small cells get segmented, which are likely artifacts. We can hence filter cells that are too small or too big. In order to figure out decent cutoffs, we want to look at the distribution of cell sizes.

In [ ]:
# checking the distribution of cell sizes
sp_object = sp_object.pp.add_observations("area")
df = sp_object.pp.get_layer_as_df('_obs')
_ = plt.hist(df['area'], bins=100)

In [ ]:
# which cutoffs would you choose based on this histogram?
# use the pp.filter_by_obs() method to filter out cells below and above your thresholds

Now, we also want to grow the masks by two pixels in each direction to try to capture cytoplasm and membrane.

In [ ]:
# TODO: use pp.grow_cells() to grow the cells. Plot the results.

## 4. Quantifying Protein Expression per Cell
Now that we have sensible segmentation masks, we can quantify the protein expression in each cell. There are multiple ways to do this, but taking the median intensity and then applying an arcsinh-transform has been proven to work pretty well.

In [ ]:
# Depending on how you implemented the previous steps, you should now have the segmentation mask in one object, but the thresholded intensities in the other.
# In order to perform quantification, we can extract the segmentation from the sp_object, and add it to the thresholded object. 
# TODO: extract the segmentation from sp_object and add it to sp_object_thresholded

In [ ]:
# TODO: on the thresholded object, add a quantification (e. g. by using the mean intensity) and transform the expression matrix using the arcsinh method

As you can see, this introduced a new layer called `_intensity`. We can now use this to predict cell types.

## 5. Cell Type Prediction
There are several ways to predict cell types. Since we thresholded our data beforehand, we can simply take the argmax of the cell type specific channels to get an idea of the cell types we are looking at. Methods related to cell type prediction are all implemented in the label (la) module.

In [ ]:
# this dictionary maps from markers to cell types
marker_ct_dict = {'PAX5': 'B', 'CD3': 'T', 'CD11b': 'Myeloid', 'CD11c': 'Dendritic', 'CD15': 'Granulo', 'CD68': 'Macro', 'Podoplanin': 'Stroma PDPN', 'CD31': 'Stroma CD31', 'CD34''Stroma CD34', 'CD90': 'Stroma CD90', 'CD56': 'NK'}

In [ ]:
# TODO: use la.predict_cell_types_argmax() to predict cell types. Investigate what the resulting object looks like.

This added a couple of things. For one, `_obs` now contains a feature called `_labels`. Furthermore, the `_la_properties` layer assigns each cell type to a color, which can be useful for plotting.

## 6. Plotting
Finally, let's do some plotting of the predicted cell types next to the markers. Before plotting, we can set some colors for the cell types (labels).

In [ ]:
# setting the colors for the cell types
# TODO: if your object is named differently, you can simply substitute this line with the appropriate variable names. You can also switch out the colors if you want a different color scheme.
sp_object_thresholded = sp_object_thresholded.la.set_label_colors(marker_ct_dict.values(), colors)

In [ ]:
# TODO: visualize the labels by using pl.show(render_labels=True)
# try to customize the plot, e. g. by also plotting the segmentation on top.
# you might need to downsample the image again due to colabs RAM limits

At this point, you should have a decent grasp on what is required to turn a multi-channel image into a single-cell expression matrix, which you can subsequently use to predict cell types.

Naturally, there are different alternatives for all of these steps. For segmentation, you could also use `stardist` or `mesmer`. If you wanted to use a more sophisticated approach for cell type prediction, you could use `astir`. In addition, you could try how different quantification and normalization techniques affect your cell type predictions. You can refer to the [spatialproteomics documentation](https://sagar87.github.io/spatialproteomics/index.html) to get an overview into some of the possible options.